# client instance 생성

In [1]:
DEPLOYMENT_URL="http://10.8.1.229:8000/"
DEPLOYMENT_URL="http://localhost:2024/"

from langgraph_sdk import get_client

client = get_client(url=DEPLOYMENT_URL)

# thread 생성 (앨런의 channel에 대응)

- 생성 요청 후 thread_id를 포함한 데이터를 return 받음
- 하나의 thread에 대해서 여러 에이전트를 호출하고 대화가능
- 현재 앨런은 한개의 channel에 대해서 한개의 agent가 1:1로 대응되므로 현재는 고려할 필요 없지만, 추후에 1개의 채널에 대해서 여러 agent를 호출가능

In [2]:
thread = await client.threads.create()
print(thread)

thread_id = thread["thread_id"]
print(thread_id)

{'thread_id': 'effa38c5-26c7-4a9d-b520-f41ff3788806', 'created_at': '2025-08-13T01:05:51.775763+00:00', 'updated_at': '2025-08-13T01:05:51.775765+00:00', 'metadata': {}, 'status': 'idle', 'config': {}, 'values': None}
effa38c5-26c7-4a9d-b520-f41ff3788806


# 에이전트 호출

- 해당 thread에서 어떤 agent를 호출할 지 assistant_id를 포함하여 요청을 보냄

## parameters
- thread_id : 위에서 생성한 thread_id
- assistant_id : 앨런의 persona_id에 대응, agent를 구분하는 id, 슬라이드 에이전트의 경우 "slide_generate_agent"
- input : 해당 agent 요청에 대한 parameter, 현재는 messages만 존재, 앨런 페이지에서 유저로 부터 받는 input을 해당 parameter에 dict로 넣어줌
- stream_mode : 어떤 event에 따라서 메시지를 받을지 지정, updates(한 단계가 끝날 때마나 메시지가 옴), events(llm이 생성하는 단위로 메시지 생성)

In [3]:
list_msg_id = list()


def get_data(chunk):
    data = list(chunk.data.values())[0]
    return data


def filter_ai_msg(chunk):
    # 유저와 채팅 창에 출력될 ai 메시지를 필터링하는 함수
    if 'updates' in chunk.event:
        data = get_data(chunk)

        if data is not None:
            if 'messages' in data.keys():
                if data['messages'][-1]["name"] is not None:
                    msg_type = data['messages'][-1]['type']
                    msg_id = data['messages'][-1]['id']
                    if msg_id not in list_msg_id and msg_type == "ai":
                        list_msg_id.append(msg_id)
                        return True
    return False


def filter_silde_data(chunk):
    # slide 관련 데이터를 담은 메시지를 필터링하는 함수
    # 슬라이드 한페이지 단위로 메시지가 옴
    if 'updates' in chunk.event:
        node_name = chunk.data.keys()
        if 'executor' in node_name:
            return True

In [4]:
assistant_id = "slide_generate_agent"
list_chunk = list()

In [5]:
input = { "messages": "안녕"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    try:
        list_chunk.append(chunk)
        if filter_ai_msg(chunk):
            print("#" * 10 + "  AI 메시지  " + "#" * 10)
            print(chunk.data['agent']['messages'][-1]['content'])
        elif filter_silde_data(chunk):
            print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
            print(chunk)
    except:
        continue
    pass

##########  AI 메시지  ##########
안녕하세요! 슬라이드 생성을 도와드릴 수 있습니다. 어떤 주제로 슬라이드를 만들고 싶으신가요?


In [6]:
input = { "messages": "제주도 여행을 주제로 슬라이드를 만들어줘"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    try:
        list_chunk.append(chunk)
        if filter_ai_msg(chunk):
            print("#" * 10 + "  AI 메시지  " + "#" * 10)
            data = get_data(chunk)
            print(data['messages'][-1]['content'])
        elif filter_silde_data(chunk):
            print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
            print(chunk)
    except:
        continue
    pass

##########  AI 메시지  ##########
좋아요! 제주도 여행을 주제로 슬라이드를 만들기 위해 몇 가지 질문을 드리겠습니다.

1. 슬라이드에 포함하고 싶은 특정 장소나 명소가 있나요?
2. 여행의 목적이 무엇인가요? 예를 들어, 휴양, 문화 탐방, 음식 탐방 등이 있을 수 있습니다.
3. 슬라이드의 청중은 누구인가요? (예: 가족, 친구, 동료 등)
4. 총 몇 장의 슬라이드를 원하시나요?
5. 슬라이드에 어떤 정보가 포함되어야 하나요? 예: 역사, 활동, 호텔 정보 등.


In [7]:
input = {"messages": "알아서 해줘"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):

    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        msg_id = data['messages'][-1]['id']

        print(data['messages'][-1]['content'])
    elif filter_silde_data(chunk):
        print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)


##########  AI 메시지  ##########

##########  AI 메시지  ##########
검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
##########  AI 메시지  ##########
생성된 목차는 다음과 같습니다. 

2. 제주도 여행 - 제주도에서 꼭 방문해야 할 여행 명소들을 소개합니다. 
3. 제주도 여행 - 제주도의 대표적인 음식과 미식 경험을 탐색합니다. 
4. 제주도 여행 - 여행 시 숙박하기 좋은 장소들을 소개합니다. 
5. 제주도 여행 - 제주도를 여행하기에 가장 적합한 시기와 날씨에 대해 알아봅니다. 
6. 제주도 여행 - 여행을 더욱 즐겁고 알차게 즐기기 위한 팁과 조언을 제공합니다. 

##########  슬라이드 데이터  ##########
##########  슬라이드 데이터  ##########
##########  슬라이드 데이터  ##########
##########  슬라이드 데이터  ##########
##########  슬라이드 데이터  ##########
##########  슬라이드 데이터  ##########
##########  슬라이드 데이터  ##########
##########  AI 메시지  ##########
제주도 여행에 대한 슬라이드를 생성하기 위해 필요한 정보를 전달했습니다. 이제 슬라이드 생성 에이전트가 적절한 슬라이드를 만들어줄 것입니다. 슬라이드에는 제주도의 주요 명소, 대표적인 음식, 숙박 정보, 여행 시기, 그리고 여행 팁 등이 포함될 것입니다. 


In [8]:
for chunk in list_chunk:
    if filter_silde_data(chunk):
        print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
        print(chunk.data)
        break

In [9]:
for chunk in list_chunk:
    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        print(data['messages'][-1]['content'])


In [10]:
state = await client.threads.get_state(thread_id=thread_id)

for msg in state["values"]["messages"]:
    if msg["type"] != "human":
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        print(msg["content"])

##########  AI 메시지  ##########
안녕하세요! 슬라이드 생성을 도와드릴 수 있습니다. 어떤 주제로 슬라이드를 만들고 싶으신가요?
##########  AI 메시지  ##########
좋아요! 제주도 여행을 주제로 슬라이드를 만들기 위해 몇 가지 질문을 드리겠습니다.

1. 슬라이드에 포함하고 싶은 특정 장소나 명소가 있나요?
2. 여행의 목적이 무엇인가요? 예를 들어, 휴양, 문화 탐방, 음식 탐방 등이 있을 수 있습니다.
3. 슬라이드의 청중은 누구인가요? (예: 가족, 친구, 동료 등)
4. 총 몇 장의 슬라이드를 원하시나요?
5. 슬라이드에 어떤 정보가 포함되어야 하나요? 예: 역사, 활동, 호텔 정보 등.
##########  AI 메시지  ##########

##########  AI 메시지  ##########
Successfully transferred to slide_generate_agent
##########  AI 메시지  ##########
검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
##########  AI 메시지  ##########
생성된 목차는 다음과 같습니다. 

2. 제주도 여행 - 제주도에서 꼭 방문해야 할 여행 명소들을 소개합니다. 
3. 제주도 여행 - 제주도의 대표적인 음식과 미식 경험을 탐색합니다. 
4. 제주도 여행 - 여행 시 숙박하기 좋은 장소들을 소개합니다. 
5. 제주도 여행 - 제주도를 여행하기에 가장 적합한 시기와 날씨에 대해 알아봅니다. 
6. 제주도 여행 - 여행을 더욱 즐겁고 알차게 즐기기 위한 팁과 조언을 제공합니다. 

##########  AI 메시지  ##########
Transferring back to supervisor
##########  AI 메시지  ##########
Successfully transferred back to supervisor
##########  AI 메시지  ##########
제주도 여행에 대한 슬라이드를 생성하기

In [11]:
len(state["values"]["slides"])

14

In [12]:
state["values"]["slides"]

[{'slide_type': 'title',
  'topic': '제주도 여행',
  'idx': 0,
  'name': '타이틀 슬라이드',
  'description': '제주도 여행에 대한 프레젠테이션 타이틀 페이지',
  'requirements': ['타이틀 텍스트', '부제목 또는 설명'],
  'research': True,
  'content': '## 타이틀 슬라이드\n\n제주도는 한국의 대표적인 여행지로, 다양한 자연경관과 문화적 명소로 유명합니다. 이 프레젠테이션에서는 제주도의 주요 관광 명소와 꼭 방문해야 할 장소들을 소개합니다. 여러분에게 숨 막히는 풍경과 독특한 문화 경험을 제공할 제주도 여행을 준비하며, 이 프레젠테이션은 귀하의 여행 계획에 도움이 될 것입니다.\n\n프레젠테이션의 타이틀 페이지는 제주도의 아름다움을 강조하기 위해 자연경관, 특히 성산일출봉이나 한라산과 같은 상징적인 장소들의 이미지로 구성되었습니다. 제주도는 유네스코 세계자연유산으로 등재된 지역으로, 이 타이틀 페이지는 이러한 자연 경관이 주는 감동을 시각적으로 전달하고자 합니다. 타이틀 페이지의 시각적 요소는 제주도의 다양한 매력을 효과적으로 전달하기 위해 감귤, 돌하르방, 해녀의 이미지들을 포함하여 보다 지역적인 색채를 더합니다 [1][2].\n\n프레젠테이션의 디자인은 깔끔하고 현대적인 느낌을 주어 제주도의 트렌디한 감성을 반영하며, 여행의 기대감과 설렘을 더할 수 있도록 합니다. 특히 색상 면에서 민트색과 노란색의 조화로운 사용을 통해 보기에도 즐거운 디자인을 제공하고자 했습니다 [3]. \n\n여러분의 편안하고 만족스러운 제주도 여행을 위한 이 프레젠테이션이 유익하기를 바랍니다. 제주도의 특별한 매력들을 상세히 알아보고, 준비된 다양한 여행 팁과 추천 루트를 통해 여러분의 여행이 더 즐겁고 풍성해질 것입니다.\n\n### Sources\n[1] Jeju Island, a Picturesque Paradise for a Peaceful Retreat - Vis